# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298233316129                   -0.85    5.6         
  2   -8.300186113623       -2.71       -1.25    1.4    155ms
  3   -8.300432637804       -3.61       -1.86    1.6    161ms
  4   -8.300462434363       -4.53       -2.71    2.8    176ms
  5   -8.300464441049       -5.70       -3.20    2.6    407ms
  6   -8.300464546208       -6.98       -3.32    5.0    268ms
  7   -8.300464589421       -7.36       -3.47    9.2    385ms
  8   -8.300464616873       -7.56       -3.62    2.1    218ms
  9   -8.300464637892       -7.68       -3.85    1.5    194ms
 10   -8.300464640338       -8.61       -3.94    1.9    664ms
 11   -8.300464643572       -8.49       -4.24    2.1    212ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67375983944                   -0.70    6.4         
  2   -16.67792940404       -2.38       -1.14    2.1    458ms
  3   -16.67919463003       -2.90       -1.86    2.2    424ms
  4   -16.67927950878       -4.07       -2.71    3.5    486ms
  5   -16.67928594677       -5.19       -3.06    3.5    534ms
  6   -16.67928619226       -6.61       -3.43    2.1    394ms
  7   -16.67928621639       -7.62       -3.92    3.1    442ms
  8   -16.67928622154       -8.29       -4.49    2.5    442ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32586760107                   -0.56    6.8         
  2   -33.33281898934       -2.16       -1.00    1.0    1.33s
  3   -33.33412926655       -2.88       -1.73    5.0    1.80s
  4   -33.33428390230       -3.81       -2.65    2.6    1.45s
  5   -33.33694237657       -2.58       -2.50    7.0    2.31s
  6   -33.33694292685       -6.26       -2.54    5.6    1.69s
  7   -33.33694370909       -6.11       -3.07    1.0    1.23s
  8   -33.33694374022       -7.51       -3.58    5.5    1.66s
  9   -33.33694377819       -7.42       -3.92    3.1    1.58s
 10   -33.33694378236       -8.38       -4.38    2.4    1.44s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298370074846                   -0.85    5.0         
  2   -8.300238787588       -2.73       -1.59    1.0    114ms
  3   -8.300429017706       -3.72       -2.56    2.4    146ms
  4   -8.300391324516   +   -4.42       -2.32    4.8    229ms
  5   -8.300464243131       -4.14       -3.44    1.0    123ms
  6   -8.300464584073       -6.47       -3.85    3.5    242ms
  7   -8.300464638550       -7.26       -4.25    1.9    177ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31782029552                   -0.56    6.2         
  2   -33.30693943239   +   -1.96       -1.27    1.2    1.21s
  3   -14.92145923830   +    1.26       -0.43    6.2    2.59s
  4   -33.31489987232        1.26       -1.58    5.5    2.30s
  5   -33.26048603560   +   -1.26       -1.38    3.0    1.83s
  6   -32.82818367154   +   -0.36       -1.21    3.4    1.80s
  7   -33.29141305073       -0.33       -1.73    3.9    1.74s
  8   -33.33552443264       -1.36       -2.23    2.8    1.44s
  9   -33.33583413590       -3.51       -2.21    2.9    1.88s
 10   -33.33689050586       -2.98       -2.90    2.0    1.42s
 11   -33.33692205166       -4.50       -3.22    2.8    1.76s
 12   -33.33693386766       -4.93      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.